In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

In [2]:
import matplotlib as mpl
from tqdm import trange
import import_ipynb

In [3]:
import style_fun as func
import Model as model

importing Jupyter notebook from style_fun.ipynb
importing Jupyter notebook from Model.ipynb


In [4]:
style_weight = 0.01
content_weight = 1e4
total_variation_weight = 1e8

epochs = 10
steps_per_epoch = 100

In [5]:
mpl.rcParams['figure.figsize']=(12,12)
mpl.rcParams['axes.grid']=False

In [6]:
content_path='style17.jpeg'
style_path='style20.jpg'

In [7]:
content_image=func.load_img(content_path)
style_image=func.load_img(style_path)

In [8]:
content_layers=['block5_conv1']
style_layers=['block1_conv1',
             'block2_conv1',
             'block3_conv1',
             'block4_conv1',
             'block5_conv1',]

In [9]:
num_content_layers=len(content_layers)
num_style_layers=len(style_layers)

In [10]:
extractor=model.StyleContentModel(style_layers,content_layers)

In [11]:
style_targets=extractor(style_image)['style']
content_targets=extractor(content_image)['content']

In [13]:
opt = tf.optimizers.Adam(learning_rate=0.02,beta_1=0.99,epsilon=1e-1)
image = tf.Variable(content_image + tf.random.truncated_normal(content_image.shape,mean=0.0,stddev=0.08),trainable=True)

In [24]:
def style_content_lose(outputs):
    style_outputs = outputs['style'] #用来表示style信息的网络层输出，这里已经计算过Gram矩阵了，已经除过NM
    content_outputs = outputs['content'] #用来表示content信息的网络层输出，不需要Gram矩阵
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_targets[name])**2) 
                          for name in style_outputs.keys()])
    style_loss *= style_weight / num_style_layers
    
    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2) 
                            for name in content_outputs.keys()])
    content_loss *= content_weight / num_content_layers
    loss = style_loss + content_loss
    return loss

In [25]:
@tf.function()
def train_step(image):
    with tf.GradientTape() as tape:
        outputs = extractor(image)
        loss = style_content_lose(outputs)
        loss += total_variation_weight*func.total_variation_loss(image)
    grad=tape.gradient(loss,image)
    opt.apply_gradients([(grad,image)])
    image.assign(func.clip_0_1(image))

In [26]:
for n in trange(epochs*steps_per_epoch):
    train_step(image)

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [1:42:56<00:00,  6.18s/it]


In [ ]:
plt.imshow(image.read_value()[0])
plt.show()
print(image.read_value()[0].shape)


In [ ]:
Eimg=tf.image.convert_image_dtype(image.read_value([0],tf.uint8))
Eimg=tf.image.ecode_jpeg(Eimg)
tf.io.write_file('F13.jpg',Eimg)